In [2]:
# import unicodecsv as csv
import csv
# conda install unicodecsv

# Data Analysis Process

### Step 1: Get data

Categorical Data - Any value that if you ran mathematical operation on, you'd get nonsensical data

Ordinal Data - Any value that if you ran mathematical operation on it, you'd get sensical data

### Step 2: Clean Data

In [3]:
# enrollment_data = []
# file = open('./enrollments.csv', 'r')
# data = csv.DictReader(file)
# for i in data:
#     enrollment_data.append(i)
# file.close()

In [4]:
# len(enrollment_data)

In [5]:
# enrollment_data[0]

In [6]:
def read_from_csv(filepath):
    with open(filepath) as file:
        data = csv.DictReader(file)
        return list(data)

enrollment_data = read_from_csv('./enrollments.csv')
engagement_data = read_from_csv('./engagements.csv')
submission_data = read_from_csv('./submissions.csv')

In [7]:
print(len(enrollment_data))
print(len(engagement_data))
print(len(submission_data))

1640
136240
3642


In [8]:
enrollment_data[0]

{'account_key': '700',
 'status': 'canceled',
 'join_date': '2017-11-10',
 'cancel_date': '2017-11-16',
 'days_to_cancel': '6',
 'is_enrolled': 'FALSE',
 'is_canceled': 'TRUE'}

In [9]:
engagement_data[0]

{'acct': '0',
 'utc_date': '2018-01-09',
 'num_courses_visited': '1',
 'total_minutes_visited': '11.6793745',
 'lessons_completed': '0',
 'projects_completed': '0'}

In [10]:
submission_data[0]

{'creation_date': '2018-01-14',
 'completion_date': '2018-01-16',
 'assigned_rating': 'UNGRADED',
 'account_key': '256',
 'lesson_key': '3176718735',
 'processing_state': 'EVALUATED'}

In [11]:
from datetime import datetime as dt

In [12]:
dt.strptime('2018-01-14', '%Y-%m-%d')

datetime.datetime(2018, 1, 14, 0, 0)

In [13]:
dt.strftime(dt(2018, 1, 14, 0, 0), '%Y-%m-%d')
dt.now()

datetime.datetime(2022, 3, 8, 15, 30, 15, 220291)

In [14]:
dt.strftime(dt.now(), '%Y/%m/%d %M:%S')

'2022/03/08 30:15'

In [15]:
def convert_to_date(string):
    if string == '':
        return None
    return dt.strptime(string, '%Y-%m-%d')

def convert_to_bool(string):
    if string == '':
        return None
    elif string == 'TRUE':
        return True
    elif string == 'FALSE':
        return False

def convert_to_int(string):
    if string == '':
        return None
    return int(string)

def convert_to_float(string):
    if string == '':
        return None
    return float(string)

In [16]:
for r in enrollment_data:
    r['join_date'] = convert_to_date(r['join_date'])
    r['cancel_date'] = convert_to_date(r['cancel_date'])
    r['days_to_cancel'] = convert_to_int(r['days_to_cancel'])
    r['is_enrolled'] = convert_to_bool(r['is_enrolled'])
    r['is_canceled'] = convert_to_bool(r['is_canceled'])

In [17]:
enrollment_data[0]

{'account_key': '700',
 'status': 'canceled',
 'join_date': datetime.datetime(2017, 11, 10, 0, 0),
 'cancel_date': datetime.datetime(2017, 11, 16, 0, 0),
 'days_to_cancel': 6,
 'is_enrolled': False,
 'is_canceled': True}

In [18]:
for r in engagement_data:
    r['utc_date'] = convert_to_date(r['utc_date'])
    r['num_courses_visited'] = convert_to_int(r['num_courses_visited'])
    r['total_minutes_visited'] = convert_to_float(r['total_minutes_visited'])
    r['lessons_completed'] = convert_to_int(r['lessons_completed'])
    r['projects_completed'] = convert_to_int(r['projects_completed'])

In [19]:
for r in submission_data:
    r['creation_date'] = convert_to_date(r['creation_date'])
    r['completion_date'] = convert_to_date(r['completion_date'])

In [20]:
# Make consistent all column labels
# RUN ONLY ONCE
for r in engagement_data:
    r['account_key'] = r['acct']
    # destroy the old 'acct' reference
    del r['acct']
# RUN ONLY ONCE

In [21]:
engagement_data[0]

{'utc_date': datetime.datetime(2018, 1, 9, 0, 0),
 'num_courses_visited': 1,
 'total_minutes_visited': 11.6793745,
 'lessons_completed': 0,
 'projects_completed': 0,
 'account_key': '0'}

In [22]:
def get_unique_records(dataset, column_name):
    unique_data = set()
    for row in dataset:
        unique_data.add(row[column_name])
    return unique_data

In [23]:
unique_enrollments = get_unique_records(enrollment_data, 'account_key')
unique_engagements = get_unique_records(engagement_data, 'account_key')
unique_submissions = get_unique_records(submission_data, 'account_key')

In [24]:
# unique_engagements

In [25]:
print(len(unique_enrollments))
print(len(unique_engagements))
print(len(unique_submissions))

1302
1237
743


In [26]:
# investigate whether an account_key exists inside of unique_engagements.
# if it does not...possible red flag

outliers = 0
for r in enrollment_data:
    if r['account_key'] not in unique_engagements and r['join_date'] != r['cancel_date']:
        outliers += 1
        print(r, end='\n\n')

{'account_key': '1304', 'status': 'canceled', 'join_date': datetime.datetime(2018, 1, 10, 0, 0), 'cancel_date': datetime.datetime(2018, 3, 10, 0, 0), 'days_to_cancel': 59, 'is_enrolled': True, 'is_canceled': True}

{'account_key': '1304', 'status': 'canceled', 'join_date': datetime.datetime(2018, 3, 10, 0, 0), 'cancel_date': datetime.datetime(2018, 6, 17, 0, 0), 'days_to_cancel': 99, 'is_enrolled': True, 'is_canceled': True}

{'account_key': '1101', 'status': 'current', 'join_date': datetime.datetime(2018, 2, 25, 0, 0), 'cancel_date': None, 'days_to_cancel': None, 'is_enrolled': True, 'is_canceled': False}



In [27]:
test_accounts = set()
for r in enrollment_data:
    if r['is_enrolled']:
        test_accounts.add(r['account_key'])
    
print(len(test_accounts))

6


In [28]:
test_accounts

{'1069', '1101', '1304', '312', '448', '818'}

In [29]:
# for any of the accounts that we found above, we need to remove their corresponding records from the other
# dataframe to prevent "screwing up" our data findings later down the road
def remove_test_accounts(dataset):
    records_to_remove = []
    for row_dict in dataset:
        if row_dict['account_key'] not in test_accounts:
            records_to_remove.append(row_dict)
    return records_to_remove

In [30]:
true_enrollments = remove_test_accounts(enrollment_data)
true_engagements = remove_test_accounts(engagement_data)
true_submissions = remove_test_accounts(submission_data)

In [31]:
print(len(true_enrollments))
print(len(true_engagements))
print(len(true_submissions))

1622
135656
3634


<h4>Find quality accounts in which:</h4>
<ol>
    <li>The account is currently active i.e. is_canceled is False</li>
    <li>days_to_cancel is greater than 7 i.e. the trial period</li>
    <li>The latest vald record for each particular VALID user</li>
</ol>

In [32]:
# {
#     '488': datetime()
# }

In [33]:
accounts_in_good_standing = dict()
for row_dict in true_enrollments:
    # if account is currently active
    if row_dict['days_to_cancel'] is not None and row_dict['cancel_date'] is not None:
        # if the difference in days_to_cancel and join_date is greater than the 7-day trial period
        if not row_dict['is_canceled'] or row_dict['days_to_cancel'] > 7:
            # grab the most recent join_date. we don't care about their previous accounts.
            if row_dict['account_key'] not in accounts_in_good_standing or row_dict['join_date'] > accounts_in_good_standing[row_dict['account_key']]:
                # create a dictionary key with the account_key number and set its value to the most recent join date
                accounts_in_good_standing[row_dict['account_key']] = row_dict['join_date']
        

In [34]:
accounts_in_good_standing

{'429': datetime.datetime(2018, 3, 10, 0, 0),
 '60': datetime.datetime(2018, 1, 14, 0, 0),
 '322': datetime.datetime(2018, 2, 12, 0, 0),
 '584': datetime.datetime(2018, 1, 14, 0, 0),
 '458': datetime.datetime(2017, 11, 10, 0, 0),
 '1058': datetime.datetime(2018, 1, 14, 0, 0),
 '45': datetime.datetime(2017, 11, 10, 0, 0),
 '315': datetime.datetime(2017, 11, 10, 0, 0),
 '408': datetime.datetime(2018, 4, 1, 0, 0),
 '51': datetime.datetime(2018, 3, 10, 0, 0),
 '323': datetime.datetime(2017, 11, 10, 0, 0),
 '130': datetime.datetime(2018, 4, 1, 0, 0),
 '550': datetime.datetime(2018, 5, 28, 0, 0),
 '44': datetime.datetime(2017, 11, 10, 0, 0),
 '440': datetime.datetime(2017, 11, 11, 0, 0),
 '57': datetime.datetime(2017, 11, 11, 0, 0),
 '1090': datetime.datetime(2017, 11, 11, 0, 0),
 '541': datetime.datetime(2017, 11, 12, 0, 0),
 '756': datetime.datetime(2017, 11, 15, 0, 0),
 '101': datetime.datetime(2017, 11, 12, 0, 0),
 '800': datetime.datetime(2018, 3, 4, 0, 0),
 '702': datetime.datetime(201

In [35]:
def remove_free_trials(dataset):
    free_trials_list = []
    for row_dict in dataset:
        if row_dict['account_key'] in accounts_in_good_standing:
            free_trials_list.append(row_dict)
    return free_trials_list

In [36]:
quality_enrollments = remove_free_trials(true_enrollments)
quality_engagements = remove_free_trials(true_engagements)
quality_submissions = remove_free_trials(true_submissions)

In [37]:
print(len(quality_enrollments))
print(len(quality_engagements))
print(len(quality_submissions))

719
55675
1110


### Step 3: Answer Questions with Data

In [37]:
# HOMEWORK:

# Take a look at data and come up with 3 good questions and use some of the methods done in this 
# Jupyter Notebook to successfully answer the questions.

In [54]:
# 1.How many lessons in total were completed by students?

lessons_completed = []
for r in quality_engagements:
    if r['lessons_completed']:
        lessons_completed.append(r['lessons_completed'])

print(sum(lessons_completed))

6071


In [88]:
# 2. How many passed, incomplete, distinction, and ungraded assignments were there?

passed = []
ungraded= []
incomplete= []
distinction=[]

for r in quality_submissions:
    if r['assigned_rating'].count('PASSED'):
        passed.append(r['assigned_rating'])
    elif r['assigned_rating'].count('UNGRADED'):
        ungraded.append(r['assigned_rating'])
    elif r['assigned_rating'].count('INCOMPLETE'):
        incomplete.append(r['assigned_rating'])
    elif r['assigned_rating'].count('DISTINCTION'):
        distinction.append(r['assigned_rating'])
        
    
print(len(passed)) 
print(len(ungraded))
print(len(incomplete))
print(len(distinction))

505
26
566
11


In [96]:
# 3. What is the average number of days to cancel?

import statistics

days_to_cancel = []

for r in quality_enrollments:
        if r['days_to_cancel']:
            days_to_cancel.append(r['days_to_cancel'])
            
print(statistics.mean(days_to_cancel))

67.89739413680782
